<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_vision_qwen_vl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [38]:
!nvidia-smi

Fri Sep 20 03:26:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# transformers qwen2-vl

In [1]:
!pip show torch torchvision torchaudio transformers autoawq

Name: torch
Version: 2.4.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, fastai, torchaudio, torchvision
---
Name: torchvision
Version: 0.19.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: fastai
---
Name: torchaudio
Version: 2.4.1+cu121
Summary: An audio package for PyTorch
Home-page: https://github.com/pytorch/audio
Author: Soumith Chintala, David Pollack, Sean Naren, Peter Goldsborough, Moto Hira, Caroline Chen, Jeff Hwang, Zhaoheng Ni, Xiaohui Zhang
Author-e

In [4]:
!pip install -q -U autoawq torch torchvision torchaudio git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.6 MB/s

In [5]:
!pip show torch torchvision torchaudio transformers autoawq

Name: torch
Version: 2.3.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, autoawq, autoawq_kernels, fastai, torchaudio, torchvision
---
Name: torchvision
Version: 0.18.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: fastai
---
Name: torcha

In [6]:
!pip install -q qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 53.6 MB/s eta 0:00:00


In [7]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from transformers import TextIteratorStreamer
import torch

from qwen_vl_utils import process_vision_info
from time import perf_counter

model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct"
model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct-Int8"
model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct-Int4"
model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct-AWQ"

model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct-Int8"
model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct-Int4"
model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct-AWQ"
model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct"


# default: Load the model on the available device(s)
#model = Qwen2VLForConditionalGeneration.from_pretrained(
#    model_ckpt_name, torch_dtype=torch.float16, device_map="cuda"
#)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# see detail: https://huggingface.co/docs/transformers/perf_infer_gpu_one
model = Qwen2VLForConditionalGeneration.from_pretrained(
     model_ckpt_name,
     torch_dtype="auto",
     #attn_implementation="sdpa", #sdpa or flash_attention_2, no eager
     #attn_implementation="flash_attention_2", #sdpa or flash_attention_2, no eager
     device_map="cuda",
)

# default processer
#processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(model_ckpt_name, min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "描述下图片内容"},
        ],
    }
]
times=[]
start_time = perf_counter()
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)


# Inference: Generation of the output with TextIteratorStreamer
streamer = TextIteratorStreamer(processor, skip_prompt=True, skip_special_tokens=True)
# Use Thread to run generation in background
# Otherwise, the process is blocked until generation is complete
# and no streaming effect can be observed.
from threading import Thread
generation_kwargs = dict(**inputs, streamer=streamer, max_new_tokens=128)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

generated_text = ""
for new_text in streamer:
    #print(new_text)
    times.append(perf_counter() - start_time)
    generated_text += new_text
    start_time = perf_counter()
print(generated_text)

print("times",times)
print("total cost:",sum(times))

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

这张图片展示了一位女性和一只狗在海滩上互动的场景。女性穿着格子衬衫和黑色裤子，坐在沙滩上，她的右腿弯曲，左腿伸直，与狗进行着高五的手势。狗坐在她的左边，看起来很友好，正在伸出前爪与女性的手进行互动。背景是广阔的海洋和天空，阳光从天空中照射下来，给整个场景增添了一种温暖的氛围。
times [11.10934917700007, 0.10860272000002169, 0.053479603000027964, 0.051345179999998436, 0.05146614700004193, 0.06373810399998092, 0.059759550999956446, 0.05206930099996043, 0.05618639799990888, 0.05894172700004674, 0.05733621999991101, 0.05407474999992701, 0.05494507799994608, 0.05267313199999535, 0.05713982899999337, 0.06204880300003879, 0.0555522289999999, 0.05801493599994956, 0.05691138400004547, 0.060380331999908776, 0.04278147000002264, 0.037267768999981854, 0.03656445500007521, 0.036699733999967066, 0.03612123400000655, 0.046668873000044186, 0.03706623799996578, 0.03666210400001546, 0.03622852000000876, 0.03696890699995947, 0.03649673699999312, 0.0361609410000483, 0.03683489800005191, 0.036474942000040755, 0.03807090199995855, 0.03644943699998748, 0.03783088199998019, 0.03798406199996407, 0.039352664000034565, 0.03837063900004978, 0.0382456

# achatbot vision lm qwen

In [22]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

Cloning into 'achatbot'...
remote: Enumerating objects: 4148, done.
remote: Counting objects: 100% (727/727), done.
remote: Compressing objects: 100% (380/380), done.
remote: Total 4148 (delta 354), reused 658 (delta 303), pack-reused 3421 (from 1)
Receiving objects: 100% (4148/4148), 6.32 MiB | 11.76 MiB/s, done.
Resolving deltas: 100% (2219/2219), done.
Submodule 'deps/ChatTTS' (https://github.com/weedge/ChatTTS.git) registered for path 'deps/ChatTTS'
Submodule 'deps/CosyVoice' (https://github.com/weedge/CosyVoice.git) registered for path 'deps/CosyVoice'
Submodule 'deps/SenseVoice' (https://github.com/weedge/SenseVoice.git) registered for path 'deps/SenseVoice'
Submodule 'ui/educator-client' (https://github.com/ai-bot-pro/educator-client.git) registered for path 'ui/educator-client'
Submodule 'ui/web-client-ui' (https://github.com/ai-bot-pro/web-client-ui.git) registered for path 'ui/web-client-ui'
Cloning into '/content/achatbot/deps/ChatTTS'...
remote: Enumerating objects: 1927, d

In [23]:
%cd /content/achatbot

/content/achatbot


In [24]:
!bash scripts/pypi_achatbot.sh dev

upload to devpypi
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 61.0
* Getting build dependencies for sdist...
running egg_info
creating pypi_build/app/achatbot.egg-info
writing pypi_build/app/achatbot.egg-info/PKG-INFO
writing dependency_links to pypi_build/app/achatbot.egg-info/dependency_links.txt
writing requirements to pypi_build/app/achatbot.egg-info/requires.txt
writing top-level names to pypi_build/app/achatbot.egg-info/top_level.txt
writing manifest file 'pypi_build/app/achatbot.egg-info/SOURCES.txt'
reading manifest file 'pypi_build/app/achatbot.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'pypi_build/app/achatbot.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing pypi_build/app/achatbot.egg-info/PKG-INFO
writing dependency_links to pypi_build/app/achatbot.egg-info/dependency_links.txt
writing requirements to pypi_build/app/achatbot.egg-info/requires.txt


In [29]:
!pip install -q "dist/achatbot-0.0.7.1-py3-none-any.whl[daily_room_audio_stream,sense_voice_asr,llm_transformers_manual_vision_qwen,tts_edge]"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 91.0 MB/s eta 0:00:00


In [30]:
!pip show transformers torch torchaudio torchvision av funasr edge-tts daily-python

Name: transformers
Version: 4.45.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autoawq
---
Name: torch
Version: 2.2.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, 

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
!huggingface-cli download Qwen/Qwen2-VL-7B-Instruct --local-dir ./models/Qwen/Qwen2-VL-7B-Instruct --local-dir-use-symlinks False

In [27]:
!huggingface-cli download Qwen/Qwen2-VL-2B-Instruct --local-dir ./models/Qwen/Qwen2-VL-2B-Instruct --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 14 files:   0% 0/14 [00:00<?, ?it/s]Downloading '.gitattributes' to 'models/Qwen/Qwen2-VL-2B-Instruct/.cache/huggingface/download/.gitattributes.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 7.97MB/s]
Download complete. Moving file to models/Qwen/Qwen2-VL-2B-Instruct/.gitattributes
Fetching 14 files:   7% 1/14 [00:00<00:03,  4.22it/s]
README.md: 100% 16.7k/16.7k [00:00<00:00, 57.4MB/s]
Download complete. Moving file to models/Qwen/Qwen2-VL-2B-Instruct/README.md

LICENSE: 100% 11.3k/11.3k [00:00<00:00, 35.7MB/s]
Download complete. Moving file to models/Qwen/Qwen2-VL-2B-Instruct/LICENSE
Fetching 14 files: 100% 14/14 [00:43<00:00,  3.11s/it]
/content/achatbot/models/Qwen/Qwen2-VL-2B-Instruct


In [ ]:
# cpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

In [ ]:
# gpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" LLM_DEVICE=cuda \
    VIDEO_FILE="/content/capture6_2024-09-15_20-10-33.mp4" \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

2024-09-16 08:34:22.321939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 08:34:22.342666: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 08:34:22.349199: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-16 08:34:22.363605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-16 08:34:23,489 - tensorflow - DEBUG 

In [ ]:
# gpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" LLM_DEVICE=cuda LLM_CHAT_HISTORY_SIZE=0 \
    VIDEO_FILE="/content/capture6_2024-09-15_20-10-33.mp4" \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-09-18 04:40:14.840128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 04:40:14.861969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 04:40:14.868381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 04:40:14.883286: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimiz

In [ ]:
!LLM_TAG=llm_transformers_manual_vision_qwen LLM_DEVICE=cuda \
    LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" \
    LLM_CHAT_HISTORY_SIZE=0 \
    python -m unittest test.integration.processors.test_vision_processor.TestVisionProcessor

2024-09-18 04:41:21.775146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 04:41:21.797616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 04:41:21.804454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 04:41:21.820325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 04:41:23.087716: W tensorflow/compiler/tf2

> [!NOTE]
>
> need add DAILY_API_KEY on colab Secret

In [ ]:
from google.colab import userdata
daily_api_key=userdata.get('DAILY_API_KEY')
#print(daily_api_key)

run bot task woker with bot.json, e.g.: daily_describe_transformers_vision_bot.json

use daily room stream, u can click bot joined the room url e.g.: https://weedge.daily.co/chat-bot to start chat with bot with audio and camera stream




In [36]:
!DAILY_API_KEY=$daily_api_key python -m src.cmd.bots.main -f /content/daily_describe_transformers_vision_bot.json

{
    "chat_bot_name": "DailyDescribeVisionBot",
    "config": {
        "asr": {
            "args": {
                "language": "zn",
                "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
            },
            "tag": "sense_voice_asr"
        },
        "llm": {
            "args": {
                "chat_history_size": 0,
                "init_chat_prompt": "\u8bf7\u7528\u4e2d\u6587\u4ea4\u6d41",
                "lm_device": "cuda",
                "lm_model_name_or_path": "./models/Qwen/Qwen2-VL-2B-Instruct",
                "model_type": "chat_completion"
            },
            "language": "zh",
            "tag": "llm_transformers_manual_vision_qwen"
        },
        "tts": {
            "args": {
                "gender": "Male",
                "language": "zh",
                "voice_name": "zh-CN-YunjianNeural"
            },
            "tag": "tts_edge"
        },
        "vad": {
            "args": {
                "stop_secs": 0.7
   